##### Problem Statement : To Predict the likely future GPS Location of a User using HMM model, Data - Geolife Trajectories of Users from China, Open Source.

In [1]:
import glob, pandas as pd, numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
#data=glob.glob('C:/Users/DVSK/Downloads/Geolife Trajectories 1.3/Geolife Trajectories 1.3/Data/004/Trajectory/*.plt')

In [3]:
#for i in data:
    
 #       with open(i,'r') as f:
    
  #          lines=f.readlines()[6:]
    
   #         with open('C:/Users/DVSK/Desktop/wow lbz/4.csv','a+') as f1:
        
    #            for line in lines:
     #               f1.write(line)

### Geolife Trajectories Data : 4 Users

In [4]:
data=glob.glob('C:/Users/DVSK/Desktop/wow labz/*.csv')

In [5]:
data

['C:/Users/DVSK/Desktop/wow labz\\0.csv',
 'C:/Users/DVSK/Desktop/wow labz\\1.csv',
 'C:/Users/DVSK/Desktop/wow labz\\2.csv',
 'C:/Users/DVSK/Desktop/wow labz\\3.csv',
 'C:/Users/DVSK/Desktop/wow labz\\4.csv']

### Data Preprocessing

In [6]:
df=pd.DataFrame()
for i in range(len(data)):
    dat=pd.read_csv(data[i],header=None)
    dat.columns=['latitude','longitude','a','b','c','Date','Time']
    dat.drop(columns=['a','b','c'],inplace=True)
    dat['Pid']=i+1
    dat['dtime']=dat.Date+' '+dat.Time
    dat['dtime+1']=list(dat['dtime'][1:])+[list(dat['dtime'])[-1]]
    dat['tdiff']=pd.to_datetime(dat['dtime+1'])-pd.to_datetime(dat.dtime)
    
    df=pd.concat([dat,df])
df.reset_index(inplace=True,drop=True)    

In [7]:
df['time_spent']=[int(i.split(' ')[0])*24*60*60  +    int(i.split(' ')[2].split(':')[0])*60*60   +  int(i.split(' ')[2].split(':')[1])*60  +  int(i.split(' ')[2].split(':')[2].split('.')[0])          for i in df.tdiff.astype(str)]

#### Columns created are 
- dtime+1 - from dtime column- next location time
- tdiff - time difference between current location time and previous location time
- time_spent - time in seconds user spent at a given location

In [8]:
df[(df.time_spent>=0) & (df.Pid==1)].sort_values('dtime')

,latitude,longitude,Date,Time,Pid,dtime,dtime+1,tdiff,time_spent
1346710,39.984702,116.318417,2008-10-23,02:53:04,1,2008-10-23 02:53:04,2008-10-23 02:53:10,00:00:06,6
1346711,39.984683,116.318450,2008-10-23,02:53:10,1,2008-10-23 02:53:10,2008-10-23 02:53:15,00:00:05,5
1346712,39.984686,116.318417,2008-10-23,02:53:15,1,2008-10-23 02:53:15,2008-10-23 02:53:20,00:00:05,5
1346713,39.984688,116.318385,2008-10-23,02:53:20,1,2008-10-23 02:53:20,2008-10-23 02:53:25,00:00:05,5
1346714,39.984655,116.318263,2008-10-23,02:53:25,1,2008-10-23 02:53:25,2008-10-23 02:53:30,00:00:05,5
...,...,...,...,...,...,...,...,...,...
1520575,40.000403,116.327255,2009-07-05,07:44:55,1,2009-07-05 07:44:55,2009-07-05 07:45:00,00:00:05,5
1520576,40.000433,116.327209,2009-07-05,07:45:00,1,2009-07-05 07:45:00,2009-07-05 07:45:05,00:00:05,5
1520577,40.000443,116.327186,2009-07-05,07:45:05,1,2009-07-05 07:45:05,2009-07-05 07:45:10,00:00:05,5
1520578,40.000522,116.327132,2009-07-05,07:45:10,1,2009-07-05 07:45:10,2009-07-05 07:45:15,00:00:05,5


- Taken User 4 data, and he/she spent more than 5 min at a given location

In [9]:
data_min=df[(df.time_spent>=300) & (df.Pid==4)].sort_values('dtime')
data_min.reset_index(drop=True,inplace=True)

### Clustering

- Clustered the data using DBSCAN with a sample size of 10, and metric - haversine for sperical distance calculation

In [10]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
coords=data_min[['latitude','longitude']]
kms_rad=6371.088
epsilon=0.1/kms_rad
db= DBSCAN(eps=epsilon, min_samples=10, algorithm="ball_tree",metric="haversine").fit(np.radians(coords))
cluster_labels=db.labels_
num_clusters=len(set(cluster_labels))
print('Clustered ' + str(len(data_min)) + ' points to ' + str(num_clusters) + ' clusters')

Clustered 1912 points to 22 clusters


In [11]:
data_min['cluster_labels']=cluster_labels

In [12]:
data_min.head()

,latitude,longitude,Date,Time,Pid,dtime,dtime+1,tdiff,time_spent,cluster_labels
0,40.008197,116.319406,2008-10-23,18:07:54,4,2008-10-23 18:07:54,2008-10-23 18:13:14,00:05:20,320,0
1,40.007738,116.318767,2008-10-23,18:16:29,4,2008-10-23 18:16:29,2008-10-24 02:02:27,07:45:58,27958,0
2,40.000018,116.327411,2008-10-24,02:13:27,4,2008-10-24 02:13:27,2008-10-24 03:36:40,01:23:13,4993,1
3,40.004361,116.322342,2008-10-24,03:45:00,4,2008-10-24 03:45:00,2008-10-24 03:54:40,00:09:40,580,0
4,40.006146,116.325746,2008-10-24,03:57:40,4,2008-10-24 03:57:40,2008-10-24 05:13:05,01:15:25,4525,2


In [13]:
len(cluster_labels)

1912

- Calculating Cluster Centroids based on Cluster_lables.

In [14]:
p=data_min.groupby('cluster_labels').agg({'latitude':np.mean}).reset_index()
q=data_min.groupby('cluster_labels').agg({'longitude':np.mean}).reset_index()

In [15]:
cc=pd.merge(p,q,'inner','cluster_labels')

In [16]:
cc.head()

,cluster_labels,latitude,longitude
0,-1,38.349288,117.266589
1,0,40.008191,116.320596
2,1,40.000230,116.327253
3,2,40.005972,116.325794
4,3,39.991185,116.327828


In [17]:
#help(hmm)

In [18]:
# unique clusters
a=len(data_min['cluster_labels'].unique())
a

22

In [19]:
X=data_min['cluster_labels'].values

In [20]:
len(X)

1912

- Took first 1900 cluster values for train and remaining 12 for test

In [21]:
len(X[:1900])

1900

### Model

In [22]:

from hmmlearn import hmm
from sklearn import preprocessing

enc = preprocessing.LabelEncoder()
classes =enc.fit(X[:1900]).transform(X[:1900])

print(classes.shape)
classes = np.reshape(classes,[len(classes),1])
model = hmm.MultinomialHMM(a)
hidden_states = model.fit(classes)  
y_pred,z= model.sample(12)
#y_sub=np.around(y_sub)
y_pred=enc.inverse_transform(y_pred)




(1900,)


- Hidden Markov Model is Used to predict the future location of user by 22 cluster lables.

In [23]:
X[1900:]

array([-1,  1,  3,  1,  0,  1,  1, -1, -1, -1, -1, -1], dtype=int64)

In [24]:
y_pred

array([ 1,  1,  1, -1, 18,  1,  3,  1,  0,  0,  0, 18], dtype=int64)

In [25]:
metrics.accuracy_score(X[1900:],y_pred)

0.16666666666666666

- Resulted with the accuracy of 16%

In [26]:
metrics.confusion_matrix(X[1900:],y_pred)

array([[0, 3, 2, 0, 1],
       [0, 0, 0, 0, 1],
       [1, 0, 2, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0]], dtype=int64)

In [27]:
metrics.f1_score(X[1900:],y_pred,average='weighted')

0.14814814814814817

- F1 Score - 14%

In [28]:
cc.head()

,cluster_labels,latitude,longitude
0,-1,38.349288,117.266589
1,0,40.008191,116.320596
2,1,40.000230,116.327253
3,2,40.005972,116.325794
4,3,39.991185,116.327828


In [29]:
import gmplot

gmap = gmplot.GoogleMapPlotter(sum(data_min.latitude)/len(data_min.latitude), sum(data_min.longitude)/len(data_min.longitude),6)
gmap.scatter(data_min.latitude, data_min.longitude,color='green', size=5,marker=True)
gmap.scatter(cc.latitude, cc.longitude,color='blue', size=30,label=['cc']*len(cc.longitude),marker=True)
gmap.scatter(cc[cc.cluster_labels==(y_pred[0])].latitude, cc[cc.cluster_labels==(y_pred[0])].longitude,color='orange', size=20,marker=True,tittle=['User 4 Next Location'],label=['4'])

gmap.draw("user004_map.html")
